In [1]:
# Import all required packages
from torch.utils.data import Dataset
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np
import torch
from typing import List
import csv

class SignLanguageMNIST(Dataset):
    """Sign Language classification dataset for A-Y without J (J and Z require movement)
    Each sample is 1 x 1 x 28 x 28, and each label is a scalar.
    """

    @staticmethod
    def get_label_mapping():
        """
        Return the label mapping which contains a list 0-24 without 9
        """
        mapping = list(range(25))
        mapping.pop(9)
        return mapping

    @staticmethod
    def read_label_samples_from_csv(path: str):
        """
        Args:
            path: Path to dataset (.csv file) 
        First column in CSV is the label and remaining 28x28 values
        are image pixel values.
        Returns:
            labels: labels got from the mapping function
            samples: all the samples in the csv file 
        """
        mapping = SignLanguageMNIST.get_label_mapping()
        labels, samples = [], []
        with open(path) as f:
            _ = next(f)  # skip header
            for line in csv.reader(f):
                label = int(line[0])
                labels.append(mapping.index(label))
                samples.append(list(map(int, line[1:])))
        return labels, samples

    def __init__(self,
            path: str="data/sign_mnist_train.csv",
            mean: List[float]=[0.485],
            std: List[float]=[0.229]):
        """
        Args:
            path: Path to dataset (.csv file) 
            mean: Mean of the dataset, has value set default
            std: Standard Deviation of the dataset, has value set default`
        """
        labels, samples = SignLanguageMNIST.read_label_samples_from_csv(path)
        self._samples = np.array(samples, dtype=np.uint8).reshape((-1, 28, 28, 1))
        self._labels = np.array(labels, dtype=np.uint8).reshape((-1, 1))

        self._mean = mean
        self._std = std

    def __len__(self):
        return len(self._labels)

    def __getitem__(self, idx):
        """
        Args:
            idx: index of the image 
        Apply transforms to convert to PILImage, apply crop,
        convert to tensor and normalize the tensor.
        Returns:
            image,label: transformes image and its respective label 
        """
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(28, scale=(0.8, 1.2)),
            transforms.ToTensor(),
            transforms.Normalize(mean=self._mean, std=self._std)])

        return {
            'image': transform(self._samples[idx]).float(),
            'label': torch.from_numpy(self._labels[idx]).float()
        }
    
    

In [2]:
def get_train_test_loaders(batch_size=32):
    """
    Args:
        batch_size: optional argument, default is 32
    Creates training and testing dataloaders.
        """
    trainset = SignLanguageMNIST('C:/Users/12134/Downloads/3258_5337_bundle_archive/sign_mnist_train.csv')
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testset = SignLanguageMNIST('C:/Users/12134/Downloads/3258_5337_bundle_archive/sign_mnist_test.csv')
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    return trainloader, testloader

In [3]:
if __name__ == '__main__':
    loader, _ = get_train_test_loaders(2)
    print(next(iter(loader)))

{'image': tensor([[[[ 0.1768,  0.2453,  0.3823,  ...,  0.8789,  0.8276,  0.8276],
          [ 0.2282,  0.3309,  0.4851,  ...,  0.9132,  0.8618,  0.8447],
          [ 0.3138,  0.4337,  0.5878,  ...,  0.9646,  0.9474,  0.8961],
          ...,
          [ 1.2899,  1.4612,  1.5639,  ..., -1.6384, -1.3302, -1.2617],
          [ 1.3242,  1.4954,  1.6324,  ..., -1.5014, -1.5528, -1.3473],
          [ 0.6563,  0.7933,  0.8789,  ..., -1.3815, -1.4672, -1.5185]]],


        [[[ 0.9474,  0.9474,  0.9646,  ...,  0.0398,  0.6221,  0.5193],
          [ 0.9988,  0.9988,  0.9988,  ..., -0.2684,  0.6392,  0.5707],
          [ 1.0502,  1.0331,  1.0331,  ..., -0.1314,  0.7419,  0.5878],
          ...,
          [-0.2171, -0.1999, -0.1486,  ..., -1.7925, -1.7240, -1.8439],
          [-0.5596, -0.5938, -0.5938,  ..., -1.7754, -1.7069, -1.9809],
          [-0.2684, -0.2513, -0.3027,  ..., -1.8439, -1.7069, -2.1008]]]]), 'label': tensor([[ 8.],
        [21.]])}


In [4]:
"""CNN based Model architecture"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 6, 3)
        self.conv3 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 48)
        self.fc3 = nn.Linear(48, 24)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
"""Train model for 20 epochs, with a learning rate of 0.01, 0.9 momentum, 
SGD optimizer over Cross Entropy Loss and save the trained model"""


import torch.optim as optim
import torch.nn.functional as F

def main():
    net = Net().float()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

    trainloader, _ = get_train_test_loaders()
    for epoch in range(20):  
        train(net, criterion, optimizer, trainloader, epoch)
    torch.save(net.state_dict(), "checkpoint.pth")


def train(net, criterion, optimizer, trainloader, epoch):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs = Variable(data['image'].float())
        labels = Variable(data['label'].long())
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels[:, 0])
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.6f' % (epoch, i, running_loss / (i + 1)))


if __name__ == '__main__':
    main()

In [5]:
from torch.utils.data import Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np


def evaluate(outputs: Variable, labels: Variable) -> float:
    """Evaluate model outputs """
    Y = labels.numpy()
    Yhat = np.argmax(outputs, axis=1)
    return float(np.sum(Yhat == Y))


def batch_evaluate(
        net: Net,
        dataloader: torch.utils.data.DataLoader) -> float:
    """Evaluate model in batches, if dataset is too large."""
    score = n = 0.0
    for batch in dataloader:
        n += len(batch['image'])
        outputs = net(batch['image'])
        if isinstance(outputs, torch.Tensor):
            outputs = outputs.detach().numpy()
        score += evaluate(outputs, batch['label'][:, 0])
    return score / n


def validate():
    trainloader, testloader = get_train_test_loaders()
    net = Net().float().eval()

    pretrained_model = torch.load("checkpoint.pth")
    net.load_state_dict(pretrained_model)

    print('=' * 10, 'PyTorch', '=' * 10)
    train_acc = batch_evaluate(net, trainloader) * 100.
    print('Training accuracy: %.1f' % train_acc)
    test_acc = batch_evaluate(net, testloader) * 100.
    print('Validation accuracy: %.1f' % test_acc)

    trainloader, testloader = get_train_test_loaders(1)


if __name__ == '__main__':
    validate()

========== PyTorch ==========
Training accuracy: 99.5
Validation accuracy: 96.8


In [ ]:
"""Testing on custom individual data"""

import cv2

index_to_letter = list('ABCDEFGHIKLMNOPQRSTUVWXY')

img = cv2.imread('C:/Users/12134/OneDrive/Pictures/Test/V.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
pixels = cv2.resize(gray,(28,28))
pixels = pixels-np.min(pixels)/(np.max(pixels)-np.min(pixels))
data = pixels.reshape(1, 1, 28, 28).astype(np.float32)

model = Net().float().eval()
pretrained_model = torch.load('checkpoint.pth')
model.load_state_dict(pretrained_model)
model.eval()
x = torch.FloatTensor(data)
y = model(Variable(x))
pred = torch.argmax(y).cpu().numpy()
print(index_to_letter[int(pred)])